In [1]:
import numpy as np
import itertools as it
from scipy import optimize
import json
import inspect
import re

def debug(x):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} = \n{}\n".format(r,x))

def flag(x):
    print("------------------------------------------")
    print(x)
    print("------------------------------------------")



def get_checks(data):
    """
    :param : "data" is the dictionnary containing the data of the loaded json file that the user upload on the API
    :returns either :
        True, ""                if the data is well defined, without errors
        False, "error message"  if the data contains a error
    """
    ##################### check every value appear (pmin, pmax, co2, load, fuels ...) in the file #####################
    ## 'load', 'fuels', 'powerplants'
    for word in ['load', 'fuels', 'powerplants']:
        if word not in data.keys():
            return False, "Variable '{}' not in file or wrongly placed".format(word)
    
    ## is fuels a dict ?
    if not type(data['fuels']) is dict:
        return False, "'fuels' is not a dict"
    
    ## does fuels have more info than "gas(euro/MWh)","kerosine(euro/MWh)","co2(euro/ton)","wind(%)" ?
    if len(data['fuels']) != 4:
        return False,'"Fuels" must have 4 values : "gas(euro/MWh)","kerosine(euro/MWh)","co2(euro/ton)","wind(%)"'
    
    
    ## does all the fuels have their info ?
    for word in ["gas(euro/MWh)","kerosine(euro/MWh)","co2(euro/ton)","wind(%)"]:
        if word not in data['fuels'].keys():
            return False, "Variable '{}' not in fuels".format(word)
    
    ## is powerplants a list ?
    if not type(data['powerplants']) is list:
        return False, "'powerplants' is not a list"
    
    ## is powerplants empty ?
    if len(data['powerplants']) == 0:
        return False, "'powerplants' is an empty list"

    ## does all the powerplant have their variables (pmin,pmax, name, type and efficiency) ?
    for i, powerplant in enumerate(data["powerplants"]):
        if not type(powerplant) is dict:
            return False, "'powerplants' is not a list of dictionnaries"
        for word in ['pmin', 'pmax', 'name', 'type', 'efficiency']:
            if word not in powerplant.keys():
                return False, "Variable '{}' not in the {}th powerplant".format(word,i+1)
    
    
    ##################### check the consistency of the values ###########################
    load = data['load']
    fuels = data['fuels']
    powerplants = data['powerplants']
    
    ## is load a positive number
    if not type(load) is int and not type(load) is float:
        return False, "'load' must be a number"
    if load <= 0:
        return False, "'load' must be non negative"
    
    ##  are fuels values numbers and are they positives
    for fuelname, fuelvalue in fuels.items():
        if not type(fuelvalue) is int and not type(fuelvalue) is float:
            return False, "'{}' must be a number".format(fuelname)
        if fuelvalue < 0:
            return False, "'{}' must be non negative".format(fuelname)
    
    ## is wind(%) a number beetween 0 and 100
    if not 0 <= fuels["wind(%)"] <= 100:
        return False, "'wind' must be in percentage (beetween 0 and 100)"
    
    
    ## for the powerplants
    for i, powerplant in enumerate(data["powerplants"]):
        # is powerplant name a string ?
        if not type(powerplant["name"]) is str:
            return False, "'name' of the {}th powerplant must be a string".format(i+1)
        # is powerplant type a string ?
        if not type(powerplant["type"]) is str:
            return False, "'type' of the {}th powerplant must be a string".format(i+1)
        # is powerplant type in gasfired, turbojet, windturbine ?
        if not powerplant["type"] in ["gasfired", "turbojet", "windturbine"]:
            return False, '"type" of the {}th powerplant must be in ["gasfired", "turbojet", "windturbine"]'.format(i+1)
        # is efficiency a number beetween 0 and 1 ?
        if not type(powerplant["efficiency"]) is int and not type(powerplant["efficiency"]) is float:
            return False, "'efficiency' must be a number"
        if not 0 <= powerplant["efficiency"] <= 1:
            return False, "'efficiency' must be beetween 0 and 1"
        # is pmin a non negative number ?
        if not type(powerplant["pmin"]) is int and not type(powerplant["pmin"]) is float:
            return False, "'pmin' must be a number"
        if not 0 <= powerplant["pmin"]:
            return False, "'pmin' must superior to 0"
        # is pmax a non negative number ?
        if not type(powerplant["pmax"]) is int and not type(powerplant["pmax"]) is float:
            return False, "'pmax' must be a number"
        if not 0 <= powerplant["pmax"]:
            return False, "'pmax' must superior to 0"
        
        #is load superior to the sum of pmax ?
        if load > get_sum_of_pmax(powerplants, fuels):
            return False, "Sum of 'pmax' ({}) too low compared to 'load' ({})".format(get_sum_of_pmax(powerplants, fuels), load)
        #is load inferior to the lower pmin ?
        if load < get_minimum_of_pmin(powerplants):
            return False, "lower 'pmin' ({}) too high compared to 'load' ({})".format(get_minimum_of_pmin(powerplants), load)
        
    return True, ""



def solver(data):
    powerplants = data["powerplants"]
    fuels = data["fuels"]
    load = data["load"]
    result = []
    #####################################################################################################
    ########################################## ONLY WINDTURBINES ########################################
    #####################################################################################################

    #we have to use the windturbine first, then gasfired and then turbojet in that order (merit order)
    #first let's calculate the maximum power that can be generated with wind turbines
    pmax_windturbines = 0 #by real, we take account of the efficiency
    for powerplant in powerplants:
        if powerplant["type"] == "windturbine":
            pmax_windturbines += powerplant["pmax"]
    debug(pmax_windturbines)
    
    #if we can cover the energy demand (load variable) with only windturbines, let's use them only. 
    if pmax_windturbines >= load:
        #no matter the order of the windturbine, we will reach the load so let's arbitrary take the first ones
        current_load = 0
        for powerplant in powerplants:
            if powerplant["type"] == "windturbine" and current_load < load:
                if current_load + powerplant["pmax"] <= load:
                    p = powerplant["pmax"]
                    current_load += p
                    result.append({"name": powerplant["name"], "p": p})
                else:
                    #it's going to be the last powerplant to be used
                    p = load - current_load
                    current_load += p
                    result.append({"name": powerplant["name"], "p": p})
            else:
                #not windturbine then no power produced here
                result.append({"name": powerplant["name"], "p": 0})
        return result

    #####################################################################################################
    ########################################## WINDTURBINES + GASFIRED ##################################
    #####################################################################################################
    # if we cannot cover the energy demand (load variable) with only windturbines, we'll use gasfired as well    
    # first let's calculate the maximum power that can be generated with wind turbines + gasfired
    pmax_gasfired = 0
    for powerplant in powerplants:
        if powerplant["type"] == "gasfired":
            pmax_gasfired += powerplant["pmax"]
    
    #if we can cover the energy demand (load variable) with only windturbines + gasfired , let's only use the 2 of them. 
    if pmax_windturbines + pmax_gasfired >= load:
        # we note p_goal_gasfired the power that only the gasfired powerplant will have to generate. The rest
        # is for the windturbines
        p_goal_gasfired = load - pmax_windturbines
        
        # Now, we need to know which gasfired powerplant to activate. We are going to try every combination of gasfired powerplants
        # We will compare every combination that can generate p_goal_gasfired and we'll choose the one with best efficiency
        
        gasfired_powerplants = []
        for powerplant in powerplants:
            if powerplant["type"] == "gasfired":
                gasfired_powerplants.append(powerplant)
        debug(gasfired_powerplants)
        gasfired_powerplants_subsets = []
        
        for L in range(1,len(gasfired_powerplants) + 1):
            for subset_gasfired_powerplants in it.combinations(gasfired_powerplants, L):
                gasfired_powerplants_subsets.append(subset_gasfired_powerplants)
        
        
        best_average_efficiency = 0
        chosen_subset = None
        for gasfired_powerplants_subset in gasfired_powerplants_subsets:
            sum_pmin = 0
            sum_pmax = 0
            efficiencies = []
            for powerplant in gasfired_powerplants_subset:
                sum_pmin += powerplant["pmin"]
                sum_pmax += powerplant["pmax"]
                efficiencies.append(powerplant["efficiency"])
            if not sum_pmin < p_goal_gasfired < sum_pmax:
                continue
            if np.mean(efficiencies) > best_average_efficiency:
                best_average_efficiency = np.mean(efficiencies)
                chosen_subset = gasfired_powerplants_subset
        
        debug(chosen_subset)
        
        if chosen_subset == None:
            # p_goal_gasfired might be lower than the lowest pmin of gasfired powerplants. 
            # In that case we will have to reduce the energy covered by the windturbines. 
            # We are going to choose the gasfired powerplant with the lower pmin, activate it, 
            # and look at the necessary windturbines to activate
            chosen_gasfired_powerplant = None
            pminmin = 10**100 # Mininimum pmin for every gasfired powerplant
            for powerplant in gasfired_powerplants:
                if powerplant["pmin"] < pminmin:
                    pminmin = powerplant["pmin"]
                    chosen_gasfired_powerplant = powerplant
            result.append({"name": chosen_gasfired_powerplant["name"], "p": pminmin})
            load_windturbines = load - pminmin #remaining load the windturbines have to generate
            
            current_load = 0
            for powerplant in powerplants:
                if powerplant["type"] == "windturbine" and current_load < load_windturbines:
                    if current_load + powerplant["pmax"] <= load_windturbines:
                        p = powerplant["pmax"]
                        current_load += p
                        result.append({"name": powerplant["name"], "p": p})
                    else:
                        #it's going to be the last powerplant to be used
                        p = load_windturbines - current_load
                        current_load += p
                        result.append({"name": powerplant["name"], "p": p})
                else:
                    #not windturbine then no power produced here
                    result.append({"name": powerplant["name"], "p": 0})
            return result
            
            
        else:
            pass
            
    

        

    return result



In [2]:
DEBUG_DATA = True
DEBUG_SOLVER = False
DEBUG_POSSIBILITIES = False

with open('../tests/payloads/payload6.json', 'r') as f:
    data = json.load(f)
    #print(get_checks(data))
    print(json.dumps(solver(data), sort_keys=True, indent=4))
    
    
    if DEBUG_DATA:
        flag(json.dumps(data, sort_keys=True, indent=2))
    

pmax_windturbines = 
186

gasfired_powerplants = 
[{'name': 'gasfiredbig2', 'type': 'gasfired', 'efficiency': 0.53, 'pmin': 100, 'pmax': 460}]

chosen_subset = 
({'name': 'gasfiredbig2', 'type': 'gasfired', 'efficiency': 0.53, 'pmin': 100, 'pmax': 460},)

[]
------------------------------------------
{
  "fuels": {
    "co2(euro/ton)": 20,
    "gas(euro/MWh)": 13.4,
    "kerosine(euro/MWh)": 50.8,
    "wind(%)": 30
  },
  "load": 480,
  "powerplants": [
    {
      "efficiency": 0.53,
      "name": "gasfiredbig2",
      "pmax": 460,
      "pmin": 100,
      "type": "gasfired"
    },
    {
      "efficiency": 1,
      "name": "windpark1",
      "pmax": 150,
      "pmin": 0,
      "type": "windturbine"
    },
    {
      "efficiency": 1,
      "name": "windpark2",
      "pmax": 36,
      "pmin": 0,
      "type": "windturbine"
    }
  ]
}
------------------------------------------


In [6]:
data = ['windturbine', 'windturbine', 'gasfired', 'windturbine', 'gasfired', 'turbojet', 'turbojet']
priorities = {0: 'windturbine', 1: 'gasfired', 2: 'turbojet'}
print(priorities)

result = sorted(data, key=lambda x: tuple(map(lambda y: priorities[y], x)))




{0: 'windturbine', 1: 'gasfired', 2: 'turbojet'}


KeyError: 'w'

TypeError: '<' not supported between instances of 'float' and 'NoneType'

In [93]:
 10**100

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000